In [1]:
from PIL import Image 
import numpy as np 
import nrrd
import os,glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2,math
import matplotlib as mpl
import requests
import pandas as pd
from tqdm import tqdm

In [2]:
# download the nrrd file from allen institute
def download_file(url, file_path):
    response = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(response.content)

file_name = 'annotation_10.nrrd'
if not os.path.exists(file_name):
    url = 'http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2017/' + file_name
    download_file(url, file_name)
    print(f"File '{file_name}' downloaded successfully.")
else:
    print(f"File '{file_name}' already exists.")

File 'annotation_10.nrrd' already exists.


In [3]:
somadata,header=nrrd.read("annotation_10.nrrd")

In [4]:
dfarea = pd.read_excel("brain_areas.xlsx",index_col=0)

In [5]:
retain_area = ["LHA","PH","DMH","V3"]
retain_ID = dfarea.loc[dfarea.region.isin(retain_area)].index.tolist()

In [6]:
mask = ~np.isin(somadata,retain_ID+[0])
somadata[mask] = 5

In [7]:
dfplot = pd.read_excel("orexin_soma_infor_cluster.xlsx",index_col=0)

In [11]:
os.mkdir("raw")

In [12]:
# fig5b
bregma = True
plot_max = math.ceil(dfplot.AP.max()/10)
plot_min = int(dfplot.AP.min()/10)

try:
    os.mkdir("raw")
except:
    print("The folder already exist")

Image_slice = int((plot_max+plot_min)/2)
image = somadata[Image_slice,:,:].astype("uint8")
# image = somadata[int(start+space/20),:,:].astype("uint8")
edges = cv2.Canny(image,10,25)
edges2 = 255-edges
# dfnew = dfplot.loc[(dfplot.AP>start*10)&(dfplot.AP<start*10+space)]
fig = plt.figure(figsize=(16,8),dpi=600)
ax = fig.add_subplot(1,1,1)
ax.imshow(edges2,cmap = 'gray')
ax.scatter(dfplot.ML/10,dfplot.DV/10,s=4,c="#ff00ff",marker = "o",alpha = 1)  
ax.axis("off")

if bregma == False:
    plt.savefig(".\\raw\\soma_all_%d.jpg"%int(Image_slice),dpi=900)
else:
    plt.savefig(".\\raw\\soma_all_%d.jpg"%int(-(Image_slice*10-5472)/100),dpi=900)
plt.close()   

The folder already exist


In [10]:
# Fig 5f
bregma = True
hue_para = "cluster"
space = 350

dic_list = {1:'#E72119',2:'#1A793A',3:'#19479C',4:'#EAE83A',5:'#6EC6CA'}

plot_max = math.ceil(dfplot.AP.max()/10)
plot_min = int(dfplot.AP.min()/10)
print(plot_max)
print(plot_min)
try:
    os.mkdir("raw")
except:
    print("The folder already exist")
for start in range(plot_min,plot_max,int(space/10)):
    image = somadata[int(start+space/20),:,:].astype("uint8")
    edges = cv2.Canny(image,10,25)
    edges2 = 255-edges
    dfnew = dfplot.loc[(dfplot.AP>start*10)&(dfplot.AP<start*10+space)]
    fig = plt.figure(figsize=(16,8),dpi=600)
    ax = fig.add_subplot(1,1,1)
    ax.imshow(edges2,cmap = 'gray')
    for sep_i in dic_list:   
        dfsubplot = dfnew.loc[dfnew.loc[:,hue_para] == sep_i]
        ax.scatter(dfsubplot.ML/10,dfsubplot.DV/10,s=4,c=dic_list[sep_i],marker = "o",alpha = 1)  
    ax.axis("off")

    if bregma == False:
        plt.savefig(".\\raw\\soma_cluster_%d.jpg"%int(start+space/20),dpi=900)
    else:
        plt.savefig(".\\raw\\soma_cluster_%d.jpg"%int(-(start*10+space/2-5472)/100),dpi=900)
    plt.close()   

741
655
The folder already exist


In [24]:
dfplot_merfish.columns = ["area","AP","DV","ML","slice"]

In [25]:
# Extended Data Fig13a
dfplot_merfish = pd.read_csv("all_orexin_soma_data.csv",index_col=0)
dfplot_merfish.columns = ["area","AP","DV","ML","slice"]
bregma = True
plot_max = math.ceil(dfplot_merfish.AP.max()/10)
plot_min = int(dfplot_merfish.AP.min()/10)

try:
    os.mkdir("raw")
except:
    print("The folder already exist")

Image_slice = int((plot_max+plot_min)/2)
image = somadata[Image_slice,:,:].astype("uint8")
# image = somadata[int(start+space/20),:,:].astype("uint8")
edges = cv2.Canny(image,10,25)
edges2 = 255-edges
# dfnew = dfplot_merfish.loc[(dfplot_merfish.AP>start*10)&(dfplot_merfish.AP<start*10+space)]
fig = plt.figure(figsize=(16,8),dpi=600)
ax = fig.add_subplot(1,1,1)
ax.imshow(edges2,cmap = 'gray')
ax.scatter(dfplot_merfish.ML/10,dfplot_merfish.DV/10,s=4,c="#ff00ff",marker = "o",alpha = 1)  
ax.axis("off")

if bregma == False:
    plt.savefig(".\\raw\\soma_all_merfish_%d.jpg"%int(Image_slice),dpi=900)
else:
    plt.savefig(".\\raw\\soma_all_merfish_%d.jpg"%int(-(Image_slice*10-5472)/100),dpi=900)
plt.close()   

The folder already exist


In [11]:
os.mkdir("crop")

In [12]:
# HY
para = (2100*3,1450*3,2500*3,1950*3)
mirrow = False
# mirrow = True

In [13]:
filelist = glob.glob(".\\raw\\*")

In [14]:
# crop image
for file in filelist:
    filetmp = Image.open(file).crop(para)
    # filetmp = Image.open(file)
    filename = file.split("\\")[-1]
    # filetmp = filetmp.rotate(-90)
    if mirrow == True:
        filetmp = filetmp.transpose(Image.FLIP_LEFT_RIGHT)
        filetmp = filetmp.rotate(-90)
    filetmp.save(os.path.join(".\\crop",filename))
    filetmp.close()

C:\Users\win7\AppData\Local\Programs\Python\Python37\Lib\site-packages\PIL\Image.py:3170: DecompressionBombWarning: Image size (103680000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [15]:
os.mkdir("edge")

In [16]:
filelist = glob.glob(".\\crop\\*")

In [17]:
# add edge
add = 8
for file in filelist:
    tmp1st = Image.open(file)
    x = int(tmp1st.size[0]+add*2)
    y = int(tmp1st.size[1]+add*2)
    result = Image.new(tmp1st.mode, (x,y),"black")
    # result = Image.new(tmp1st.mode, (x,y),"white")
    result.paste(tmp1st,box=(add,add))
    result.save(".\\edge\\%s"%(file.split("\\")[-1]))
    result.close()
